In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=41cb51348429c805cb72a95b96102748a2ca4206e62825ef4491a3c4377d4e42
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Importing libraries
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc=SparkContext()

In [4]:
spark = SparkSession.builder.master("Loacal[1]").appName("admit_regress").getOrCreate()

**Prediction of chance of admission**

In [5]:
df_admit = spark.read.option("header",True).option("inferSchema",True).csv("/content/Admission_Predict.csv")

In [6]:
df_admit.show(10)

+----------+---------+-----------+-----------------+---+----+----+--------+-----+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Admit|
+----------+---------+-----------+-----------------+---+----+----+--------+-----+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1| 0.92|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1| 0.76|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1| 0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|  0.8|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0| 0.65|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|  0.9|
|         7|      321|        109|                3|3.0| 4.0| 8.2|       1| 0.75|
|         8|      308|        101|                2|3.0| 4.0| 7.9|       0| 0.68|
|         9|      302|        102|                1|2.0| 1.5| 8.0|       0|  0.5|
|        10|    

In [7]:
# To print the data types of columns
df_admit.printSchema()

root
 |-- Serial No.: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR : double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Admit: double (nullable = true)



In [8]:
df_admit.columns

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Admit']

In [9]:
df_admit.count()

400

In [10]:
len(df_admit.columns)

9

In [11]:
df_admit.createOrReplaceTempView("admit_tab")

In [15]:
df1 = spark.sql("select `GRE Score`,`TOEFL Score`,Admit from admit_tab where Admit>.85")

In [16]:
df1.show()

+---------+-----------+-----+
|GRE Score|TOEFL Score|Admit|
+---------+-----------+-----+
|      337|        118| 0.92|
|      330|        115|  0.9|
|      328|        116| 0.94|
|      334|        119| 0.95|
|      336|        119| 0.97|
|      340|        120| 0.94|
|      338|        118| 0.91|
|      340|        114|  0.9|
|      331|        112| 0.94|
|      320|        110| 0.88|
|      332|        117| 0.87|
|      326|        113| 0.91|
|      322|        110| 0.88|
|      329|        114| 0.86|
|      339|        119| 0.89|
|      332|        118| 0.94|
|      336|        112| 0.96|
|      321|        111| 0.93|
|      340|        120| 0.96|
|      320|        110| 0.92|
+---------+-----------+-----+
only showing top 20 rows



In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [18]:
x = df_admit.drop("Serial No.","Admit")

In [19]:
x.show()

+---------+-----------+-----------------+---+----+----+--------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|
+---------+-----------+-----------------+---+----+----+--------+
|      337|        118|                4|4.5| 4.5|9.65|       1|
|      324|        107|                4|4.0| 4.5|8.87|       1|
|      316|        104|                3|3.0| 3.5| 8.0|       1|
|      322|        110|                3|3.5| 2.5|8.67|       1|
|      314|        103|                2|2.0| 3.0|8.21|       0|
|      330|        115|                5|4.5| 3.0|9.34|       1|
|      321|        109|                3|3.0| 4.0| 8.2|       1|
|      308|        101|                2|3.0| 4.0| 7.9|       0|
|      302|        102|                1|2.0| 1.5| 8.0|       0|
|      323|        108|                3|3.5| 3.0| 8.6|       0|
|      325|        106|                3|3.5| 4.0| 8.4|       1|
|      327|        111|                4|4.0| 4.5| 9.0|       1|
|      328|        112|  

In [20]:
va = VectorAssembler(inputCols=x.columns,outputCol="features")

In [21]:
df_admit=va.transform(df_admit)

In [22]:
df_admit.show()

+----------+---------+-----------+-----------------+---+----+----+--------+-----+--------------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Admit|            features|
+----------+---------+-----------+-----------------+---+----+----+--------+-----+--------------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1| 0.92|[337.0,118.0,4.0,...|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1| 0.76|[324.0,107.0,4.0,...|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1| 0.72|[316.0,104.0,3.0,...|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|  0.8|[322.0,110.0,3.0,...|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0| 0.65|[314.0,103.0,2.0,...|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|  0.9|[330.0,115.0,5.0,...|
|         7|      321|        109|                3|3.0| 4.0| 8.2|       

In [23]:
df_admit.select("features").show()

+--------------------+
|            features|
+--------------------+
|[337.0,118.0,4.0,...|
|[324.0,107.0,4.0,...|
|[316.0,104.0,3.0,...|
|[322.0,110.0,3.0,...|
|[314.0,103.0,2.0,...|
|[330.0,115.0,5.0,...|
|[321.0,109.0,3.0,...|
|[308.0,101.0,2.0,...|
|[302.0,102.0,1.0,...|
|[323.0,108.0,3.0,...|
|[325.0,106.0,3.0,...|
|[327.0,111.0,4.0,...|
|[328.0,112.0,4.0,...|
|[307.0,109.0,3.0,...|
|[311.0,104.0,3.0,...|
|[314.0,105.0,3.0,...|
|[317.0,107.0,3.0,...|
|[319.0,106.0,3.0,...|
|[318.0,110.0,3.0,...|
|[303.0,102.0,3.0,...|
+--------------------+
only showing top 20 rows



In [24]:
# Rename the target variable as label
df_admit=df_admit.withColumnRenamed("Admit","label")

In [25]:
df_admit.show()

+----------+---------+-----------+-----------------+---+----+----+--------+-----+--------------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|label|            features|
+----------+---------+-----------+-----------------+---+----+----+--------+-----+--------------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1| 0.92|[337.0,118.0,4.0,...|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1| 0.76|[324.0,107.0,4.0,...|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1| 0.72|[316.0,104.0,3.0,...|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|  0.8|[322.0,110.0,3.0,...|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0| 0.65|[314.0,103.0,2.0,...|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|  0.9|[330.0,115.0,5.0,...|
|         7|      321|        109|                3|3.0| 4.0| 8.2|       

In [26]:
df_new = df_admit.select("features","label")
df_new.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[337.0,118.0,4.0,...| 0.92|
|[324.0,107.0,4.0,...| 0.76|
|[316.0,104.0,3.0,...| 0.72|
|[322.0,110.0,3.0,...|  0.8|
|[314.0,103.0,2.0,...| 0.65|
|[330.0,115.0,5.0,...|  0.9|
|[321.0,109.0,3.0,...| 0.75|
|[308.0,101.0,2.0,...| 0.68|
|[302.0,102.0,1.0,...|  0.5|
|[323.0,108.0,3.0,...| 0.45|
|[325.0,106.0,3.0,...| 0.52|
|[327.0,111.0,4.0,...| 0.84|
|[328.0,112.0,4.0,...| 0.78|
|[307.0,109.0,3.0,...| 0.62|
|[311.0,104.0,3.0,...| 0.61|
|[314.0,105.0,3.0,...| 0.54|
|[317.0,107.0,3.0,...| 0.66|
|[319.0,106.0,3.0,...| 0.65|
|[318.0,110.0,3.0,...| 0.63|
|[303.0,102.0,3.0,...| 0.62|
+--------------------+-----+
only showing top 20 rows



In [27]:
# Splitting of the dataset
train,test = df_new.randomSplit([.80,.20],seed=10)

In [28]:
train.count()

324

In [29]:
test.count()

76

In [30]:
# Modeling
from pyspark.ml.regression import LinearRegression

In [31]:
model = LinearRegression(featuresCol="features",labelCol="label")

In [32]:
model=model.fit(train)

In [33]:
predict_train= model.transform(train)

In [34]:
predict_train.show(10)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[290.0,100.0,1.0,...| 0.47| 0.4895045719083633|
|[293.0,97.0,2.0,2...| 0.64| 0.5837197272852845|
|[294.0,95.0,1.0,1...| 0.49|0.47445664641883845|
|[295.0,93.0,1.0,2...| 0.46|0.42573076112568575|
|[295.0,99.0,2.0,2...| 0.57| 0.5268403051844546|
|[295.0,101.0,2.0,...| 0.69| 0.5353886253686992|
|[296.0,95.0,2.0,3...| 0.44|0.49711688061011516|
|[296.0,99.0,2.0,2...| 0.61| 0.5608211225647326|
|[296.0,99.0,2.0,3...| 0.47| 0.4938602833092849|
|[297.0,96.0,2.0,2...| 0.43| 0.5113138680936795|
+--------------------+-----+-------------------+
only showing top 10 rows



In [35]:
predict_test=model.transform(test)

In [36]:
predict_test.show(10)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[290.0,104.0,4.0,...| 0.45| 0.5194002672137588|
|[294.0,93.0,1.0,1...| 0.46|0.44606652221556353|
|[295.0,96.0,2.0,1...| 0.47|0.46276683387795425|
|[296.0,97.0,2.0,1...| 0.49| 0.5215618454858684|
|[296.0,101.0,1.0,...|  0.6|  0.533586826949197|
|[297.0,98.0,2.0,2...| 0.59| 0.5285743135610352|
|[299.0,106.0,2.0,...| 0.64| 0.6602783290212286|
|[300.0,99.0,1.0,3...| 0.36| 0.4269896945569578|
|[300.0,102.0,2.0,...| 0.56|  0.555195727298651|
|[300.0,102.0,3.0,...| 0.63|  0.594153183247139|
+--------------------+-----+-------------------+
only showing top 10 rows



In [37]:
from pyspark.ml.evaluation import RegressionEvaluator

In [38]:
eval1= RegressionEvaluator()

In [39]:
eval1= eval1.evaluate(predict_test,{eval1.metricName:"r2"})

In [40]:
eval1

0.8279039233279797

In [41]:
eval2= RegressionEvaluator()

In [42]:
eval2=eval2.evaluate(predict_train,{eval2.metricName:"r2"})

In [43]:
eval2

0.7937730086844993

**Accuracy score of training dataset is 79 while accuracy of test data set is 83**